In [1]:
from transformers import SiglipForImageClassification, AutoImageProcessor
import os
import pandas as pd
from PIL import Image
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

2025-05-16 20:10:23.520675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747426223.869677      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747426223.965441      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
! apt-get install -y gdown

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package gdown


In [3]:
# download zip from google drive and unzip contents
! gdown --id 1rbISVuHbT_AJPw4wv4ywLym3TLVuGLDH      

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1rbISVuHbT_AJPw4wv4ywLym3TLVuGLDH
From (redirected): https://drive.google.com/uc?id=1rbISVuHbT_AJPw4wv4ywLym3TLVuGLDH&confirm=t&uuid=74640b90-3dca-40e9-9600-6ce3477a6ecc
To: /kaggle/working/frames_and_annotations.zip
100%|███████████████████████████████████████| 1.14G/1.14G [00:10<00:00, 113MB/s]


In [4]:
# Unzip the file
import zipfile
with zipfile.ZipFile("./frames_and_annotations.zip", "r") as zip_ref:
    zip_ref.extractall("frames")


In [5]:
model = SiglipForImageClassification.from_pretrained("prithivMLmods/Human-Action-Recognition")
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Human-Action-Recognition")

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
df = pd.read_csv("./frames/frames_annotations.csv")

# fix backslash to forward slash in relative paths
df["image"] = df["image"].apply(lambda x: os.path.join("frames", x.replace("\\", "/")))

#assign labels
df["label"] = df["label"].map({"not_taking_medication": 0, "taking_medication": 1})

# generate dataset from df
dataset = Dataset.from_pandas(df)

def preprocess(example):
    image = Image.open(example["image"]).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    inputs["labels"] = example["label"]
    return {
        "pixel_values": inputs["pixel_values"].squeeze(),
        "labels": inputs["labels"]
    }

dataset = dataset.map(preprocess, remove_columns=["image", "label"])

Map:   0%|          | 0/850 [00:00<?, ? examples/s]

In [7]:
dataset = dataset.train_test_split(test_size=0.1)

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Ensure logits are numpy arrays
    if isinstance(logits, tuple):
        print("logits is tuple")
        logits = logits[0]
    if isinstance(logits, torch.Tensor):
        print("logits is tensor")
        logits = logits.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor):
        print("labels is tensor")
        labels = labels.detach().cpu().numpy()
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [12]:
model.config.label2id = {"not_taking_medication": 0, "taking_medication": 1}
model.config.id2label = {0: "not_taking_medication", 1: "taking_medication"}

training_args = TrainingArguments(
    output_dir="./HAR-medication-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    hub_model_id="Adekiii/HAR-medication-finetuned",
    report_to=["none"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

In [19]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SiglipForImageClassification(
  (vision_model): SiglipVisionTransformer(
    (embeddings): SiglipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
      (position_embedding): Embedding(196, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attn): SiglipAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_feat

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.455900,0.292818,0.905882,0.000000,0.000000,0.000000
2,0.539300,0.363245,0.905882,0.000000,0.000000,0.000000
3,0.334300,0.287595,0.905882,0.000000,0.000000,0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=288, training_loss=0.5068342975444264, metrics={'train_runtime': 2705.1499, 'train_samples_per_second': 0.848, 'train_steps_per_second': 0.106, 'total_flos': 1.935798167938683e+17, 'train_loss': 0.5068342975444264, 'epoch': 3.0})

In [14]:
trainer.save_model("./HAR-med-finetunedv2")
processor.save_pretrained("./HAR-med-finetunedv2")

['./HAR-med-finetuned/preprocessor_config.json']

In [15]:
from huggingface_hub import HfApi

access_token = "..."

api = HfApi(token=access_token)
api.upload_folder(
    folder_path="HAR-med-finetunedv2",
    repo_id="tam6/HAR-medication-finetunedv2", # change with own repo
    repo_type="model",
)

model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tam6/HAR-medication-finetuned/commit/9cd7e903282c41001795d212e66f6ec0a2974967', commit_message='Upload folder using huggingface_hub', commit_description='', oid='9cd7e903282c41001795d212e66f6ec0a2974967', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tam6/HAR-medication-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='tam6/HAR-medication-finetuned'), pr_revision=None, pr_num=None)

In [7]:
# Load the processor and model from the HuggingFace Hub
processor = AutoImageProcessor.from_pretrained("Adekiii/HAR-medication-finetuned")
model = SiglipForImageClassification.from_pretrained("Adekiii/HAR-medication-finetuned")

preprocessor_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

RuntimeError: Error(s) in loading state_dict for Linear:
	size mismatch for bias: copying a param with shape torch.Size([15]) from checkpoint, the shape in current model is torch.Size([2]).

In [ ]:
test_data = dataset["test"]
print(test_data)

In [ ]:
inputs = processor(images=test_data["pixel_values"], return_tensors="pt")

# Run inference
predicted_labels = []
true_labels = test_data["labels"]

with torch.no_grad():
    #for im in test_data["pixel_values"]
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    #predicted_labels.append(predicted_class_idx)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_class_idx, average="binary", zero_division=0)


In [19]:
print("f1 score on test set: " + str(f1))

NameError: name 'f1' is not defined